# Sections


- <a href="#SHP">Reading in Shapefiles</a><br>
- <a href="#ER">enter section</a><br>
- <a href="#agg">enter section</a><br>
    -  <a href="#baseline">enter subsec</a><br>
- <a href="#match">Matching Electoral Precincts to 2022 Districts</a><br>

## Importing Packages, Reading in Shapefiles

In [1]:
import pandas as pd
import geopandas as gpd
import numpy as np
from fiona.crs import from_epsg
from shapely.geometry import Point, Polygon
import matplotlib.pyplot as plt
from matplotlib import cm
%matplotlib inline

In [2]:
#importing helper functions from helper module
from helper_module import centroid_match

### Reading in 2022 Legislative District Shapefiles

In [3]:
#2022 Texas State Legislative Districts, Upper
new_senate = gpd.read_file('data/raw/maps/tx_sldu_2022/PLANS2168.shp')
new_senate = new_senate.to_crs(epsg=3857)

In [50]:
#2022 Texas State Legislative Districts, Lower
new_house = gpd.read_file('data/raw/maps/tx_sldl_2022/PLANH2316.shp')
new_house = new_house.to_crs(epsg=3857)

In [63]:
#Checking data frames
for item in [new_senate, new_house]:
    item.columns = ['district', 'geometry'] #setting column names
    print('Number of rows is ' + str(item.shape[0])) #should be 31 for senate, 150 for house

Number of rows is 31
Number of rows is 150


In [64]:
new_senate.info() #checking for data types, and null values

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 31 entries, 0 to 30
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   district  31 non-null     int64   
 1   geometry  31 non-null     geometry
dtypes: geometry(1), int64(1)
memory usage: 624.0 bytes


In [65]:
new_house.info() #checking for data types, and null values

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   district  150 non-null    int64   
 1   geometry  150 non-null    geometry
dtypes: geometry(1), int64(1)
memory usage: 2.5 KB


### Cleaning 2020 Election Returns

In [6]:
tx_2020 = gpd.read_file('data/raw/elec_ret/tx_vest_20/tx_vest_20.shp')
tx_2020 = tx_2020.to_crs(epsg=3857)

In [7]:
#empty columns for 2022 state legislative districts
tx_2020['SD'] = 0
tx_2020['HD'] = 0

#calculating two-party vote totals for state-wide races in each precinct
tx_2020['G20PRETOT'] = tx_2020['G20PRERTRU'] + tx_2020['G20PREDBID'] #US Presidential Race
tx_2020['G20USSTOT'] = tx_2020['G20USSRCOR'] + tx_2020['G20USSDHEG'] #US Senate Race
tx_2020['G20RRCTOT'] = tx_2020['G20RRCRWRI'] + tx_2020['G20RRCDCAS'] #Texas Rail Road Commissioner

#slimming dataframe to state-wide races
tx_2020 = tx_2020.loc[:, ['CNTY','PCTKEY',
                          'G20PRERTRU','G20PRETOT',
                          'G20USSRCOR','G20USSTOT',
                          'G20RRCRWRI','G20RRCTOT',
                          'SD','HD','geometry']]

#renaming columns for readability
tx_2020.columns = ['county_key','prec_key',
                   'pre_20_r', 'pre_20_tot',
                   'uss_20_r', 'uss_20_tot',
                   'rrc_20_r','rrc_20_tot',
                   'SD','HD','geometry']
tx_2020.columns

Index(['county_key', 'prec_key', 'pre_20_r', 'pre_20_tot', 'uss_20_r',
       'uss_20_tot', 'rrc_20_r', 'rrc_20_tot', 'SD', 'HD', 'geometry'],
      dtype='object')

In [8]:
#checking number of precincts
tx_2020.shape

(9014, 11)

### Cleaning 2018 Election Returns

In [ ]:
tx_2018 = gpd.read_file('data/raw/elec_ret/tx_vest_18/tx_vest_18.shp')
tx_2018 = tx_2018.to_crs(epsg=3857)

In [ ]:
#Creating an empty column to designate 2022 State Senate District
# and 2022 State House District
tx_2018['SD'] = np.nan
tx_2018['HD'] = np.nan

#Creating two-party vote totals for each of the state-wide races
tx_2018['G18USSTOT'] = tx_2018['G18USSRCRU'] + tx_2018['G18USSDORO'] #US Senate Race
tx_2018['G18GOVTOT'] = tx_2018['G18GOVRABB'] + tx_2018['G18GOVDVAL'] #TX Gov Race
tx_2018['G18LTGTOT'] = tx_2018['G18LTGRPAT'] + tx_2018['G18LTGDCOL'] #TX Lt Gov Race
tx_2018['G18ATGTOT'] = tx_2018['G18ATGRPAX'] + tx_2018['G18ATGDNEL'] #TX Attorney Gen race
tx_2018['G18COMTOT'] = tx_2018['G18COMRHEG'] + tx_2018['G18COMDCHE'] #TX Comptroller Race
tx_2018['G18LANTOT'] = tx_2018['G18LANRBUS'] + tx_2018['G18LANDSUA'] #TX Com of Land Race
tx_2018['G18AGRTOT'] = tx_2018['G18AGRRMIL'] + tx_2018['G18AGRDOLS'] #TX Com of Ag
tx_2018['G18RRCTOT'] = tx_2018['G18RRCRCRA'] + tx_2018['G18RRCDMCA'] #TX Railroad Commissioner

#trimming dataframe to state-wide races
col_to_keep = ['CNTY','PCTKEY',
               'G18USSRCRU','G18USSTOT',
               'G18GOVRABB','G18GOVTOT',
               'G18LTGRPAT','G18LTGTOT',
               'G18ATGRPAX','G18ATGTOT',
               'G18COMRHEG','G18COMTOT',
               'G18LANRBUS','G18LANTOT',
               'G18AGRRMIL','G18AGRTOT',
               'G18RRCRCRA','G18RRCTOT',
               'SD','HD','geometry']
tx_2018 = tx_2018.loc[:, col_to_keep ]

#renaming columns
tx_2018.columns = ['county_key','prec_key',
                   'uss_18_r','uss_18_tot',
                   'gov_18_r','gov_18_tot',
                   'ltg_18_r','ltg_18_tot',
                   'atg_18_r','atg_18_tot',
                   'com_18_r','com_18_tot',
                   'lan_18_r','lan_18_tot',
                   'agr_18_r','agr_18_tot',
                   'rrc_18_r','rrc_18_tot',
                   'SD','HD','geometry']
tx_2018.columns

In [66]:
#checking number of precincts
tx_2018.shape

(8936, 21)

### Cleaning 2016 Election Returns

In [105]:
tx_2016 = gpd.read_file('data/raw/elec_ret/tx_vest_16/tx_vest_16.shp')
tx_2016 = tx_2016.to_crs(epsg=3857)

In [109]:
tx_2016.columns

Index(['CNTY', 'COLOR', 'PREC', 'PCTKEY', 'CNTYKEY', 'G16VR', 'G16SSVR',
       'G16PRERTRU', 'G16PREDCLI', 'G16PRELJOH', 'G16PREGSTE', 'G16PREOWRI',
       'G16RRCRCHR', 'G16RRCDYAR', 'G16RRCLMIL', 'G16RRCGSAL', 'G16SSCRLEH',
       'G16SSCDWES', 'G16SSCLGLA', 'G16SSCGMUN', 'G16SSCRGRE', 'G16SSCDGAR',
       'G16SSCLOXF', 'G16SSCGWAT', 'G16SSCRGUZ', 'G16SSCDROB', 'G16SSCLFUL',
       'G16SSCGCHI', 'G16SCCRKEE', 'G16SCCDMEY', 'G16SCCLASH', 'G16SCCGREP',
       'G16SCCRWAL', 'G16SCCDJOH', 'G16SCCLSTR', 'G16SCCGSAN', 'G16SCCRKEA',
       'G16SCCDBUR', 'G16SCCLBEN', 'geometry'],
      dtype='object')

In [106]:
tx_2016.head()


#[x for x.lower() in df.columns if "16" in x and "prr" in x]

,CNTY,COLOR,PREC,PCTKEY,CNTYKEY,G16VR,G16SSVR,G16PRERTRU,G16PREDCLI,G16PRELJOH,...,G16SCCLASH,G16SCCGREP,G16SCCRWAL,G16SCCDJOH,G16SCCLSTR,G16SCCGSAN,G16SCCRKEA,G16SCCDBUR,G16SCCLBEN,geometry
0,439,6,1611,4391611,220,24,45.8,4,8,1,...,1,0,4,8,1,0,4,8,1,"POLYGON ((-10835052.539 3857290.232, -10835152..."
1,27,2,0311,270311,14,2729,7.4,1406,269,46,...,74,11,1385,246,45,20,1369,257,68,"POLYGON ((-10841335.522 3663705.944, -10841379..."
2,35,5,0001,350001,18,675,5.2,402,61,6,...,17,1,378,65,10,3,370,71,15,"POLYGON ((-10891701.135 3772729.466, -10891619..."
3,201,5,0736,2010736,101,40,77.5,1,16,0,...,1,0,0,13,0,1,1,14,0,"POLYGON ((-10614383.912 3475911.233, -10614397..."
4,439,3,3039,4393039,220,3696,3.7,1780,782,99,...,87,28,1957,640,69,29,1968,625,95,"POLYGON ((-10811811.701 3893667.078, -10811813..."


In [ ]:
tx_2016['SD'] = np.nan
tx_2016['HD'] = np.nan

#calculating two-party vote totals for state-wide races in each precinct
tx_2016['G16PRETOT'] = tx_2016['G16PRERTRU'] + tx_2016['G16PREDCLI'] #US Presidential Race
tx_2016['G16RRCTOT'] = tx_2016['G16RRCRCHR'] + tx_2016['G16RRCDYAR'] #Texas Rail Road Commissioner

#slimming dataframe to state-wide races
tx_2016 = tx_2016.loc[:, ['CNTY','PCTKEY',
                          'G16PRERTRU','G16PRETOT',
                          'G16RRCRCHR','G16RRCTOT',
                          'SD','HD','geometry']]

#renaming columns for readability
tx_2016.columns = ['county_key','prec_key',
                   'pre_16_r', 'pre_16_tot',
                   'rrc_16_r','rrc_16_tot',
                   'SD','HD','geometry']
tx_2016.columns

In [67]:
#checking number of precincts
tx_2016.shape

(8832, 9)

## Matching past Election returns to 2022 legislative districts

In [68]:
#Testing the function on one precinct
test = tx_2020.iloc[0,-1].centroid
centroid_match(test, new_senate)

16

In [71]:
# using helper function to match precinct centroid from past election returns to new senate and house districts
count = 1
for item in [tx_2016, tx_2018, tx_2020]:
    for (index, geometry) in item.iterrows():
        centroid = item.loc[index]['geometry'].centroid
        item.loc[index, 'SD'] = centroid_match(centroid, new_senate)
        item.loc[index, 'HD'] = centroid_match(centroid, new_house)
        print(count, index)
    count += 1

1 0
1 1
1 2
1 3
1 4
1 5
1 6
1 7
1 8
1 9
1 10
1 11
1 12
1 13
1 14
1 15
1 16
1 17
1 18
1 19
1 20
1 21
1 22
1 23
1 24
1 25
1 26
1 27
1 28
1 29
1 30
1 31
1 32
1 33
1 34
1 35
1 36
1 37
1 38
1 39
1 40
1 41
1 42
1 43
1 44
1 45
1 46
1 47
1 48
1 49
1 50
1 51
1 52
1 53
1 54
1 55
1 56
1 57
1 58
1 59
1 60
1 61
1 62
1 63
1 64
1 65
1 66
1 67
1 68
1 69
1 70
1 71
1 72
1 73
1 74
1 75
1 76
1 77
1 78
1 79
1 80
1 81
1 82
1 83
1 84
1 85
1 86
1 87
1 88
1 89
1 90
1 91
1 92
1 93
1 94
1 95
1 96
1 97
1 98
1 99
1 100
1 101
1 102
1 103
1 104
1 105
1 106
1 107
1 108
1 109
1 110
1 111
1 112
1 113
1 114
1 115
1 116
1 117
1 118
1 119
1 120
1 121
1 122
1 123
1 124
1 125
1 126
1 127
1 128
1 129
1 130
1 131
1 132
1 133
1 134
1 135
1 136
1 137
1 138
1 139
1 140
1 141
1 142
1 143
1 144
1 145
1 146
1 147
1 148
1 149
1 150
1 151
1 152
1 153
1 154
1 155
1 156
1 157
1 158
1 159
1 160
1 161
1 162
1 163
1 164
1 165
1 166
1 167
1 168
1 169
1 170
1 171
1 172
1 173
1 174
1 175
1 176
1 177
1 178
1 179
1 180
1 181
1 182
1 183
1 184


1 1332
1 1333
1 1334
1 1335
1 1336
1 1337
1 1338
1 1339
1 1340
1 1341
1 1342
1 1343
1 1344
1 1345
1 1346
1 1347
1 1348
1 1349
1 1350
1 1351
1 1352
1 1353
1 1354
1 1355
1 1356
1 1357
1 1358
1 1359
1 1360
1 1361
1 1362
1 1363
1 1364
1 1365
1 1366
1 1367
1 1368
1 1369
1 1370
1 1371
1 1372
1 1373
1 1374
1 1375
1 1376
1 1377
1 1378
1 1379
1 1380
1 1381
1 1382
1 1383
1 1384
1 1385
1 1386
1 1387
1 1388
1 1389
1 1390
1 1391
1 1392
1 1393
1 1394
1 1395
1 1396
1 1397
1 1398
1 1399
1 1400
1 1401
1 1402
1 1403
1 1404
1 1405
1 1406
1 1407
1 1408
1 1409
1 1410
1 1411
1 1412
1 1413
1 1414
1 1415
1 1416
1 1417
1 1418
1 1419
1 1420
1 1421
1 1422
1 1423
1 1424
1 1425
1 1426
1 1427
1 1428
1 1429
1 1430
1 1431
1 1432
1 1433
1 1434
1 1435
1 1436
1 1437
1 1438
1 1439
1 1440
1 1441
1 1442
1 1443
1 1444
1 1445
1 1446
1 1447
1 1448
1 1449
1 1450
1 1451
1 1452
1 1453
1 1454
1 1455
1 1456
1 1457
1 1458
1 1459
1 1460
1 1461
1 1462
1 1463
1 1464
1 1465
1 1466
1 1467
1 1468
1 1469
1 1470
1 1471
1 1472
1 1473
1 1474

1 2522
1 2523
1 2524
1 2525
1 2526
1 2527
1 2528
1 2529
1 2530
1 2531
1 2532
1 2533
1 2534
1 2535
1 2536
1 2537
1 2538
1 2539
1 2540
1 2541
1 2542
1 2543
1 2544
1 2545
1 2546
1 2547
1 2548
1 2549
1 2550
1 2551
1 2552
1 2553
1 2554
1 2555
1 2556
1 2557
1 2558
1 2559
1 2560
1 2561
1 2562
1 2563
1 2564
1 2565
1 2566
1 2567
1 2568
1 2569
1 2570
1 2571
1 2572
1 2573
1 2574
1 2575
1 2576
1 2577
1 2578
1 2579
1 2580
1 2581
1 2582
1 2583
1 2584
1 2585
1 2586
1 2587
1 2588
1 2589
1 2590
1 2591
1 2592
1 2593
1 2594
1 2595
1 2596
1 2597
1 2598
1 2599
1 2600
1 2601
1 2602
1 2603
1 2604
1 2605
1 2606
1 2607
1 2608
1 2609
1 2610
1 2611
1 2612
1 2613
1 2614
1 2615
1 2616
1 2617
1 2618
1 2619
1 2620
1 2621
1 2622
1 2623
1 2624
1 2625
1 2626
1 2627
1 2628
1 2629
1 2630
1 2631
1 2632
1 2633
1 2634
1 2635
1 2636
1 2637
1 2638
1 2639
1 2640
1 2641
1 2642
1 2643
1 2644
1 2645
1 2646
1 2647
1 2648
1 2649
1 2650
1 2651
1 2652
1 2653
1 2654
1 2655
1 2656
1 2657
1 2658
1 2659
1 2660
1 2661
1 2662
1 2663
1 2664

1 3706
1 3707
1 3708
1 3709
1 3710
1 3711
1 3712
1 3713
1 3714
1 3715
1 3716
1 3717
1 3718
1 3719
1 3720
1 3721
1 3722
1 3723
1 3724
1 3725
1 3726
1 3727
1 3728
1 3729
1 3730
1 3731
1 3732
1 3733
1 3734
1 3735
1 3736
1 3737
1 3738
1 3739
1 3740
1 3741
1 3742
1 3743
1 3744
1 3745
1 3746
1 3747
1 3748
1 3749
1 3750
1 3751
1 3752
1 3753
1 3754
1 3755
1 3756
1 3757
1 3758
1 3759
1 3760
1 3761
1 3762
1 3763
1 3764
1 3765
1 3766
1 3767
1 3768
1 3769
1 3770
1 3771
1 3772
1 3773
1 3774
1 3775
1 3776
1 3777
1 3778
1 3779
1 3780
1 3781
1 3782
1 3783
1 3784
1 3785
1 3786
1 3787
1 3788
1 3789
1 3790
1 3791
1 3792
1 3793
1 3794
1 3795
1 3796
1 3797
1 3798
1 3799
1 3800
1 3801
1 3802
1 3803
1 3804
1 3805
1 3806
1 3807
1 3808
1 3809
1 3810
1 3811
1 3812
1 3813
1 3814
1 3815
1 3816
1 3817
1 3818
1 3819
1 3820
1 3821
1 3822
1 3823
1 3824
1 3825
1 3826
1 3827
1 3828
1 3829
1 3830
1 3831
1 3832
1 3833
1 3834
1 3835
1 3836
1 3837
1 3838
1 3839
1 3840
1 3841
1 3842
1 3843
1 3844
1 3845
1 3846
1 3847
1 3848

1 4887
1 4888
1 4889
1 4890
1 4891
1 4892
1 4893
1 4894
1 4895
1 4896
1 4897
1 4898
1 4899
1 4900
1 4901
1 4902
1 4903
1 4904
1 4905
1 4906
1 4907
1 4908
1 4909
1 4910
1 4911
1 4912
1 4913
1 4914
1 4915
1 4916
1 4917
1 4918
1 4919
1 4920
1 4921
1 4922
1 4923
1 4924
1 4925
1 4926
1 4927
1 4928
1 4929
1 4930
1 4931
1 4932
1 4933
1 4934
1 4935
1 4936
1 4937
1 4938
1 4939
1 4940
1 4941
1 4942
1 4943
1 4944
1 4945
1 4946
1 4947
1 4948
1 4949
1 4950
1 4951
1 4952
1 4953
1 4954
1 4955
1 4956
1 4957
1 4958
1 4959
1 4960
1 4961
1 4962
1 4963
1 4964
1 4965
1 4966
1 4967
1 4968
1 4969
1 4970
1 4971
1 4972
1 4973
1 4974
1 4975
1 4976
1 4977
1 4978
1 4979
1 4980
1 4981
1 4982
1 4983
1 4984
1 4985
1 4986
1 4987
1 4988
1 4989
1 4990
1 4991
1 4992
1 4993
1 4994
1 4995
1 4996
1 4997
1 4998
1 4999
1 5000
1 5001
1 5002
1 5003
1 5004
1 5005
1 5006
1 5007
1 5008
1 5009
1 5010
1 5011
1 5012
1 5013
1 5014
1 5015
1 5016
1 5017
1 5018
1 5019
1 5020
1 5021
1 5022
1 5023
1 5024
1 5025
1 5026
1 5027
1 5028
1 5029

...

1 2111
1 2112
1 2113
1 2114
1 2115
1 2116
1 2117
1 2118
1 2119
1 2120
1 2121
1 2122
1 2123
1 2124
1 2125
1 2126
1 2127
1 2128
1 2129
1 2130
1 2131
1 2132
1 2133
1 2134
1 2135
1 2136
1 2137
1 2138
1 2139
1 2140
1 2141
1 2142
1 2143
1 2144
1 2145
1 2146
1 2147
1 2148
1 2149
1 2150
1 2151
1 2152
1 2153
1 2154
1 2155
1 2156
1 2157
1 2158
1 2159
1 2160
1 2161
1 2162
1 2163
1 2164
1 2165
1 2166
1 2167
1 2168
1 2169
1 2170
1 2171
1 2172
1 2173
1 2174
1 2175
1 2176
1 2177
1 2178
1 2179
1 2180
1 2181
1 2182
1 2183
1 2184
1 2185
1 2186
1 2187
1 2188
1 2189
1 2190
1 2191
1 2192
1 2193
1 2194
1 2195
1 2196
1 2197
1 2198
1 2199
1 2200
1 2201
1 2202
1 2203
1 2204
1 2205
1 2206
1 2207
1 2208
1 2209
1 2210
1 2211
1 2212
1 2213
1 2214
1 2215
1 2216
1 2217
1 2218
1 2219
1 2220
1 2221
1 2222
1 2223
1 2224
1 2225
1 2226
1 2227
1 2228
1 2229
1 2230
1 2231
1 2232
1 2233
1 2234
1 2235
1 2236
1 2237
1 2238
1 2239
1 2240
1 2241
1 2242
1 2243
1 2244
1 2245
1 2246
1 2247
1 2248
1 2249
1 2250
1 2251
1 2252
1 2253

1 3291
1 3292
1 3293
1 3294
1 3295
1 3296
1 3297
1 3298
1 3299
1 3300
1 3301
1 3302
1 3303
1 3304
1 3305
1 3306
1 3307
1 3308
1 3309
1 3310
1 3311
1 3312
1 3313
1 3314
1 3315
1 3316
1 3317
1 3318
1 3319
1 3320
1 3321
1 3322
1 3323
1 3324
1 3325
1 3326
1 3327
1 3328
1 3329
1 3330
1 3331
1 3332
1 3333
1 3334
1 3335
1 3336
1 3337
1 3338
1 3339
1 3340
1 3341
1 3342
1 3343
1 3344
1 3345
1 3346
1 3347
1 3348
1 3349
1 3350
1 3351
1 3352
1 3353
1 3354
1 3355
1 3356
1 3357
1 3358
1 3359
1 3360
1 3361
1 3362
1 3363
1 3364
1 3365
1 3366
1 3367
1 3368
1 3369
1 3370
1 3371
1 3372
1 3373
1 3374
1 3375
1 3376
1 3377
1 3378
1 3379
1 3380
1 3381
1 3382
1 3383
1 3384
1 3385
1 3386
1 3387
1 3388
1 3389
1 3390
1 3391
1 3392
1 3393
1 3394
1 3395
1 3396
1 3397
1 3398
1 3399
1 3400
1 3401
1 3402
1 3403
1 3404
1 3405
1 3406
1 3407
1 3408
1 3409
1 3410
1 3411
1 3412
1 3413
1 3414
1 3415
1 3416
1 3417
1 3418
1 3419
1 3420
1 3421
1 3422
1 3423
1 3424
1 3425
1 3426
1 3427
1 3428
1 3429
1 3430
1 3431
1 3432
1 3433

1 4469
1 4470
1 4471
1 4472
1 4473
1 4474
1 4475
1 4476
1 4477
1 4478
1 4479
1 4480
1 4481
1 4482
1 4483
1 4484
1 4485
1 4486
1 4487
1 4488
1 4489
1 4490
1 4491
1 4492
1 4493
1 4494
1 4495
1 4496
1 4497
1 4498
1 4499
1 4500
1 4501
1 4502
1 4503
1 4504
1 4505
1 4506
1 4507
1 4508
1 4509
1 4510
1 4511
1 4512
1 4513
1 4514
1 4515
1 4516
1 4517
1 4518
1 4519
1 4520
1 4521
1 4522
1 4523
1 4524
1 4525
1 4526
1 4527
1 4528
1 4529
1 4530
1 4531
1 4532
1 4533
1 4534
1 4535
1 4536
1 4537
1 4538
1 4539
1 4540
1 4541
1 4542
1 4543
1 4544
1 4545
1 4546
1 4547
1 4548
1 4549
1 4550
1 4551
1 4552
1 4553
1 4554
1 4555
1 4556
1 4557
1 4558
1 4559
1 4560
1 4561
1 4562
1 4563
1 4564
1 4565
1 4566
1 4567
1 4568
1 4569
1 4570
1 4571
1 4572
1 4573
1 4574
1 4575
1 4576
1 4577
1 4578
1 4579
1 4580
1 4581
1 4582
1 4583
1 4584
1 4585
1 4586
1 4587
1 4588
1 4589
1 4590
1 4591
1 4592
1 4593
1 4594
1 4595
1 4596
1 4597
1 4598
1 4599
1 4600
1 4601
1 4602
1 4603
1 4604
1 4605
1 4606
1 4607
1 4608
1 4609
1 4610
1 4611

1 5649
1 5650
1 5651
1 5652
1 5653
1 5654
1 5655
1 5656
1 5657
1 5658
1 5659
1 5660
1 5661
1 5662
1 5663
1 5664
1 5665
1 5666
1 5667
1 5668
1 5669
1 5670
1 5671
1 5672
1 5673
1 5674
1 5675
1 5676
1 5677
1 5678
1 5679
1 5680
1 5681
1 5682
1 5683
1 5684
1 5685
1 5686
1 5687
1 5688
1 5689
1 5690
1 5691
1 5692
1 5693
1 5694
1 5695
1 5696
1 5697
1 5698
1 5699
1 5700
1 5701
1 5702
1 5703
1 5704
1 5705
1 5706
1 5707
1 5708
1 5709
1 5710
1 5711
1 5712
1 5713
1 5714
1 5715
1 5716
1 5717
1 5718
1 5719
1 5720
1 5721
1 5722
1 5723
1 5724
1 5725
1 5726
1 5727
1 5728
1 5729
1 5730
1 5731
1 5732
1 5733
1 5734
1 5735
1 5736
1 5737
1 5738
1 5739
1 5740
1 5741
1 5742
1 5743
1 5744
1 5745
1 5746
1 5747
1 5748
1 5749
1 5750
1 5751
1 5752
1 5753
1 5754
1 5755
1 5756
1 5757
1 5758
1 5759
1 5760
1 5761
1 5762
1 5763
1 5764
1 5765
1 5766
1 5767
1 5768
1 5769
1 5770
1 5771
1 5772
1 5773
1 5774
1 5775
1 5776
1 5777
1 5778
1 5779
1 5780
1 5781
1 5782
1 5783
1 5784
1 5785
1 5786
1 5787
1 5788
1 5789
1 5790
1 5791

KeyboardInterrupt: 

In [95]:
datasets = [(2016, tx_2016), (2018, tx_2018), (2020, tx_2020)]
#Checking whether matching worked
for year,item in datasets:
    for dist in ['HD', 'SD']:
        print(year, '\n', 'There are',item[dist].nunique(), 'in', dist)
        print('There are ', item[dist].isnull().values.sum(), 'null values')

2016 
 There are 150 in HD
There are  0 null values
2016 
 There are 31 in SD
There are  0 null values
2018 
 There are 150 in HD
There are  0 null values
2018 
 There are 31 in SD
There are  0 null values
2020 
 There are 150 in HD
There are  0 null values
2020 
 There are 31 in SD
There are  0 null values


In [89]:
#removing rows with null values
for year,item in datasets:
    for dist in ['HD', 'SD']:
        if item[dist].isnull().values.any():
            item.dropna(axis = 0, how = 'any', inplace = True)

In [102]:
tx_2020.head().drop('geometry', axis = 1)

,county_key,prec_key,pre_20_r,pre_20_tot,uss_20_r,uss_20_tot,rrc_20_r,rrc_20_tot,SD,HD
0,113,1131104,221,1394,232,1361,208,1354,16.0,107.0
1,201,2010312,1124,2584,1185,2521,1157,2457,17.0,133.0
2,351,3510003,412,440,395,427,391,420,3.0,11.0
3,181,1810304,1290,1966,1293,1935,1331,1921,30.0,62.0
4,201,2010877,1352,3906,1377,3804,1329,3742,7.0,135.0


In [104]:
#saving cleaned and matched election returns
for year,item in datasets:
    item.drop('geometry', axis = 1).to_csv(r'data/cleaned/' + 'ctx_' + str(year) + '.csv', index = False)

In [39]:
#tx_2020.dropna?
tx_2020.dropna(axis = 0, how = 'any', inplace = True)

In [41]:
tx_2020.shape

(9012, 11)

## Cleaning 2018 Election Returns


In [47]:
for (index, geometry) in tx_2018.iterrows():
    centroid = tx_2018.loc[index]['geometry'].centroid
    tx_2018.loc[index, 'SD'] = centroid_match(centroid, new_senate)
    tx_2018.loc[index, 'HD'] = centroid_match(centroid, new_house)
    print(index)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

1870
1871
1872
1873
1874
1875
1876
1877
1878
1879
1880
1881
1882
1883
1884
1885
1886
1887
1888
1889
1890
1891
1892
1893
1894
1895
1896
1897
1898
1899
1900
1901
1902
1903
1904
1905
1906
1907
1908
1909
1910
1911
1912
1913
1914
1915
1916
1917
1918
1919
1920
1921
1922
1923
1924
1925
1926
1927
1928
1929
1930
1931
1932
1933
1934
1935
1936
1937
1938
1939
1940
1941
1942
1943
1944
1945
1946
1947
1948
1949
1950
1951
1952
1953
1954
1955
1956
1957
1958
1959
1960
1961
1962
1963
1964
1965
1966
1967
1968
1969
1970
1971
1972
1973
1974
1975
1976
1977
1978
1979
1980
1981
1982
1983
1984
1985
1986
1987
1988
1989
1990
1991
1992
1993
1994
1995
1996
1997
1998
1999
2000
2001
2002
2003
2004
2005
2006
2007
2008
2009
2010
2011
2012
2013
2014
2015
2016
2017
2018
2019
2020
2021
2022
2023
2024
2025
2026
2027
2028
2029
2030
2031
2032
2033
2034
2035
2036
2037
2038
2039
2040
2041
2042
2043
2044
2045
2046
2047
2048
2049
2050
2051
2052
2053
2054
2055
2056
2057
2058
2059
2060
2061
2062
2063
2064
2065
2066
2067
2068
2069


3527
3528
3529
3530
3531
3532
3533
3534
3535
3536
3537
3538
3539
3540
3541
3542
3543
3544
3545
3546
3547
3548
3549
3550
3551
3552
3553
3554
3555
3556
3557
3558
3559
3560
3561
3562
3563
3564
3565
3566
3567
3568
3569
3570
3571
3572
3573
3574
3575
3576
3577
3578
3579
3580
3581
3582
3583
3584
3585
3586
3587
3588
3589
3590
3591
3592
3593
3594
3595
3596
3597
3598
3599
3600
3601
3602
3603
3604
3605
3606
3607
3608
3609
3610
3611
3612
3613
3614
3615
3616
3617
3618
3619
3620
3621
3622
3623
3624
3625
3626
3627
3628
3629
3630
3631
3632
3633
3634
3635
3636
3637
3638
3639
3640
3641
3642
3643
3644
3645
3646
3647
3648
3649
3650
3651
3652
3653
3654
3655
3656
3657
3658
3659
3660
3661
3662
3663
3664
3665
3666
3667
3668
3669
3670
3671
3672
3673
3674
3675
3676
3677
3678
3679
3680
3681
3682
3683
3684
3685
3686
3687
3688
3689
3690
3691
3692
3693
3694
3695
3696
3697
3698
3699
3700
3701
3702
3703
3704
3705
3706
3707
3708
3709
3710
3711
3712
3713
3714
3715
3716
3717
3718
3719
3720
3721
3722
3723
3724
3725
3726


5171
5172
5173
5174
5175
5176
5177
5178
5179
5180
5181
5182
5183
5184
5185
5186
5187
5188
5189
5190
5191
5192
5193
5194
5195
5196
5197
5198
5199
5200
5201
5202
5203
5204
5205
5206
5207
5208
5209
5210
5211
5212
5213
5214
5215
5216
5217
5218
5219
5220
5221
5222
5223
5224
5225
5226
5227
5228
5229
5230
5231
5232
5233
5234
5235
5236
5237
5238
5239
5240
5241
5242
5243
5244
5245
5246
5247
5248
5249
5250
5251
5252
5253
5254
5255
5256
5257
5258
5259
5260
5261
5262
5263
5264
5265
5266
5267
5268
5269
5270
5271
5272
5273
5274
5275
5276
5277
5278
5279
5280
5281
5282
5283
5284
5285
5286
5287
5288
5289
5290
5291
5292
5293
5294
5295
5296
5297
5298
5299
5300
5301
5302
5303
5304
5305
5306
5307
5308
5309
5310
5311
5312
5313
5314
5315
5316
5317
5318
5319
5320
5321
5322
5323
5324
5325
5326
5327
5328
5329
5330
5331
5332
5333
5334
5335
5336
5337
5338
5339
5340
5341
5342
5343
5344
5345
5346
5347
5348
5349
5350
5351
5352
5353
5354
5355
5356
5357
5358
5359
5360
5361
5362
5363
5364
5365
5366
5367
5368
5369
5370


6818
6819
6820
6821
6822
6823
6824
6825
6826
6827
6828
6829
6830
6831
6832
6833
6834
6835
6836
6837
6838
6839
6840
6841
6842
6843
6844
6845
6846
6847
6848
6849
6850
6851
6852
6853
6854
6855
6856
6857
6858
6859
6860
6861
6862
6863
6864
6865
6866
6867
6868
6869
6870
6871
6872
6873
6874
6875
6876
6877
6878
6879
6880
6881
6882
6883
6884
6885
6886
6887
6888
6889
6890
6891
6892
6893
6894
6895
6896
6897
6898
6899
6900
6901
6902
6903
6904
6905
6906
6907
6908
6909
6910
6911
6912
6913
6914
6915
6916
6917
6918
6919
6920
6921
6922
6923
6924
6925
6926
6927
6928
6929
6930
6931
6932
6933
6934
6935
6936
6937
6938
6939
6940
6941
6942
6943
6944
6945
6946
6947
6948
6949
6950
6951
6952
6953
6954
6955
6956
6957
6958
6959
6960
6961
6962
6963
6964
6965
6966
6967
6968
6969
6970
6971
6972
6973
6974
6975
6976
6977
6978
6979
6980
6981
6982
6983
6984
6985
6986
6987
6988
6989
6990
6991
6992
6993
6994
6995
6996
6997
6998
6999
7000
7001
7002
7003
7004
7005
7006
7007
7008
7009
7010
7011
7012
7013
7014
7015
7016
7017


8460
8461
8462
8463
8464
8465
8466
8467
8468
8469
8470
8471
8472
8473
8474
8475
8476
8477
8478
8479
8480
8481
8482
8483
8484
8485
8486
8487
8488
8489
8490
8491
8492
8493
8494
8495
8496
8497
8498
8499
8500
8501
8502
8503
8504
8505
8506
8507
8508
8509
8510
8511
8512
8513
8514
8515
8516
8517
8518
8519
8520
8521
8522
8523
8524
8525
8526
8527
8528
8529
8530
8531
8532
8533
8534
8535
8536
8537
8538
8539
8540
8541
8542
8543
8544
8545
8546
8547
8548
8549
8550
8551
8552
8553
8554
8555
8556
8557
8558
8559
8560
8561
8562
8563
8564
8565
8566
8567
8568
8569
8570
8571
8572
8573
8574
8575
8576
8577
8578
8579
8580
8581
8582
8583
8584
8585
8586
8587
8588
8589
8590
8591
8592
8593
8594
8595
8596
8597
8598
8599
8600
8601
8602
8603
8604
8605
8606
8607
8608
8609
8610
8611
8612
8613
8614
8615
8616
8617
8618
8619
8620
8621
8622
8623
8624
8625
8626
8627
8628
8629
8630
8631
8632
8633
8634
8635
8636
8637
8638
8639
8640
8641
8642
8643
8644
8645
8646
8647
8648
8649
8650
8651
8652
8653
8654
8655
8656
8657
8658
8659
